In [1]:
import numpy as np

# Load sudokus
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

very_easy_puzzle.npy has been loaded into the variable sudoku
sudoku.shape: (15, 9, 9), sudoku[0].shape: (9, 9), sudoku.dtype: int8

First sudoku:
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]] 

Solution of first sudoku:
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]


## Part One
You should write all of your code for solving sudokus below this cell.

You must include a function called `sudoku_solver(sudoku)` which takes one sudoku puzzle (a 9x9 NumPy array) as input, and returns the solved sudoku as another 9x9 NumPy array. This is the function which will be tested. 

In [107]:
import numpy as np

#(i) taking inspiration from a suggestion made in the Q&A, using deep copy 
import copy

#(ii) for additional functions
import itertools


                                                        #(1) Helper Functions-

def sudoku_solver(sudoku):
    
    class constraint_solver:

        #(1.0) initialising solver
        def __init__(self):
            #initialises a legal set of values for the domain
            self.dom = {}

            #initialises a legal set of variable pairs
            self.constraint = {}

            #initialises a legal list of variable names used in the constraint satisfaction solver 
            self.var = []

            #checks the number of times backtrack function has been called (and how often said function returns failure)
            self.check_call = 0
            self.check_fail = 0


        #(1.1) helper function that adds any newly established constraining values to already established constraining variables
        def add_var(self, val, dom):
            self.dom[val] = list(dom)
            self.constraint[val] = {}
            self.var.append(val)


        #(1.2) helper function that returns a tupled list of all legal pairs
        def legal_pairs(self, a, b):
            return itertools.prod(a,b)


        #(1.3) helper function that returns a tupled list of all legal constraint satisfactions, given the currently defined constraints
        def legal_constraints(self):
            return[(i,j) for i in self.constraint 
                         for j in self.constraint[i]]


        #(1.4) helper function that returns a tupled list of all legal constraint satisfactions to & from a given variable, given the currently defined constraints
        def relational_constraints(self, var):
            return [(i,var) for i in self.constraints[var]]


        #(1.5) helper function that adds any newly established constraining values between paris of values (i->j & j->i) to already established constraining variables
        def paired_constraints (self, i, j, func_filter):
            if not j in self.constraint[i]:
                self.constraint[i][j] = self.legal_pairs(self.dom[i], self.dom[j])

            #to check the variables j->i the function 'func_filter' filters all remaining, listed, legal pairs
            self.constraint[i][j] = filter(lambda x: func_filter(*x), self.constraint[i][j])


        #(1.6) helper function that gives constraints by ensuring that no two expressions in the list of variables can hold equivilent values at any given point
        def all_diff_constraints(self, var):
              for(i, j) in self.legal_pairs(var, var):
                if i != j:
                    self.paired_constraints(i, j, lambda x, y: x != y)



                                                            #(2) Implementation of AC-3 Algorithm (see README.txt for elaboration)-

        #(2.1) function that initialises the constraint solver, using deep copy of dictionary of domains, to ensure no unintentional assignment
        def assign_backtracker(self):
            assign = copy.deepcopy(self.dom)

            # Check identified constraits and eliminate non-arc consitent (AC) values
            self.infer(assign, self.legal_constraints())
            # Call 'recursuve_backtracker' (listed below) with partial assignment 'assign'
            return self.backtrack(assign)


        #(2.2) function that implements a backtracking algorithm, recursively calling possible legal values for unaassigned cells
        def recursive_backtracker(self, assign):

            #set count to  continue until all variables have been assigned a value (lists of length one)
            self.count_assigned += 1

            #check for partial assgiement 'part_assign' (variables that may hold a single value)
            part_assign = []
            for val in assign.vals():
                if len(val) != 1:
                    part_assign.append(val)

            #if list empty (i.e. not partially assigned), return assignment
            if not part_assign:
                return assign

            #identify first unassigned variable in input problem set (see definition for 'select_not_assign_var' in next function)
            not_assign_var = self.select_not_assign_var(assign)

            #each unassigned variable that value that can be assigneda value is copied and given a value 'val'
            for val in assign[not_assign_var]:
                copy_assign = copy.deepcopy(assign)
                copy_assign[not_assign_var] = [val]

                #check related legal constraints (arcs in 'A'C-3)
                if self.infer(assign_copy, self.relational_constraints(not_assign_var)):
                    result = self.recursive_backtracker(copy_assign)

                    #if the backtrack helper-function 'recursive_backtracker' does not return False, then the algorithm has found a legal solution
                    if result:
                        return result

            #else, if nothing can be returned, algorithm has failed to find solutiom
            self.check_fail += 1
            return False


        #(2.3) function that returns the first found value of a given unassigned variables, that have more than one potential legal value
        def select_not_assign_var(self, assign):

            for i in assign:
                if len(assign[i]) > 1:
                    return i

            return False


        #(2.4) function that takes variables with partial assignments (containing lists of potential legal values for undecided variables) and maps a queue of arcs that still need to be checked
        def infer(self, assign, mapped_queue):

            while mapped_queue:
                contraining_arcs = mapped_queue.pop()

                # Check whether arc satisfies identified constraints
                if self.revise(assign, contraining_arcs[0], contraining_arcs[1]):
                    if not assign[contraining_arcs[0]]:
                        return False

                    # Check all surrounding cells and add to mapped_queue if does not exist yet
                    for x in self.relational_constraints(arc[0]):
                        if x[0] not in contraining_arcs:
                            mapped_queue.append((x[0], contraining_arcs[0]))

            return True


        #(2.5) function that takes arcs that still need to be visited and removes previously legal values (in arc 'i') that do not satisfy constraints between arcs ('i' & 'j')
        def checker(self, assign, i, j):

            checked = False

            #for every arc 'i' that still needs to be visited
            for x in assign[i]:
                is_legal = True

                #for every arc 'j' that still needs to be visited
                for y in assignment[j]:

                    #if contraints include current arc value, do not remove
                    if (x, y) in self.constraint[i][j]:
                        is_legal = False

                #if a checked arc 'j' does not exist in identified constraints, remove it from list
                if is_legal:
                    assign[i].remove(x)
                    checked = True

            return checked



    #(3.0)...
    def create_sudoku_constraint_solver(sudoku, constraint_solver):

        
        

        for row in range(9):
            for col in range(9):
                if sudoku[row][col] == '0':
                    constraint_solver.add_var('%d-%d' % (row, col), map(str, range(1, 10)))
                else:
                    constraint_solver.add_var('%d-%d' % (row, col), [ sudoku[row][col] ])

        for row in range(9):
            constraint_solver.relational_constraints([ '%d-%d' % (row, col) for col in range(9) ])

        for col in range(9):
            constraint_solver.relational_constraints([ '%d-%d' % (row, col) for row in range(9) ])

        for box_row in range(3):
            for box_col in range(3):
                cells = []
                for row in range(box_row * 3, (box_row + 1) * 3):
                    for col in range(box_col * 3, (box_col + 1) * 3):
                        cells.append('%d-%d' % (row, col))
                constraint_solver.relational_constraints(cells)

        return constraint_solver

    

    #(4)...
    def print_sudoku_solution(solution):
        for row in range(9):
            for col in range(9):
                print (solution['%d-%d' % (row, col)][0]),
                if col == 2 or col == 5:
                    print ('|'),
            print
            if row == 2 or row == 5:
                print ('------+-------+------')
    
    constraint_solver = constraint_solver()
    return create_sudoku_constraint_solver(sudoku,constraint_solver)
    #(5)...
    # if __name__ == '__main__':
    #     print_sudoku_solution(constraint_solver.recursive_backtracker())

    #     # Number of times the 'recursive_backtracker' function was called
    #     print ("\nTotal backtracks: " + str(constraint_solver.check_call))

    #     # Number of times the 'recursive_backtracker' function returned failure (false)
    #     print ("Failed backtracks: " + str(constraint_solver.check_fail))



All of your code must go above this cell. You may add additional cells into the notebook if you wish, but do not duplicate or copy/paste cells as this can interfere with the grading script.

### Testing Details
There are four difficulties of sudoku provided: very easy, easy, medium, and hard. There are 15 sample sudokus in each category, with solutions as well. Difficulty was determined using reference solvers, but your code may vary; it is conceivable that your code will find some sudokus much easier or harder within a given category, or even between categories.

*All categories that are easy and above will contain* ***invalid initial states***, that is, sudoku puzzles with no solution. In this case, your function should return a 9x9 NumPy array whose values are all equal to -1.

When we test your code, we will firstly test it on the *same* very easy puzzles that you have been given. Then we will test it on additional *hidden* sudokus from each difficulty in turn, easy and up. Grades are awarded based on whether your code can solve the puzzles. For high grades on the hard puzzles, execution time will also be a factor. 

All puzzles must take under 30 seconds each on the test machine to count as successful, but you should be aiming for an average of under a second per puzzle. Hardware varies, but all tests will take place on the same modern desktop machine. Our ‘standard constraint satisfaction’ implementation takes about 0.001 seconds per puzzle for the very easy category, but struggles to solve some of the hard puzzles within the time limit.

***The hard sudokus are labelled as hard for a reason.*** We expect most submissions will not be able to solve them in a reasonable length of time. Use the stop button (■) on the toolbar if you need to terminate your code because it is taking too long.

The best way to improve the performance of your code is through a detailed understanding and smart choice of AI algorithms. This assignment is ***not*** meant to test your ability to write multi-threaded code or any other kind of high-performance code optimisations. 

#### Test Cell
The following code will run your solution over the provided sudoku puzzles. To enable it, set the constant `SKIP_TESTS` to `False`. If you fail any tests of one difficulty, the code will stop, but you can modify this behaviour if you like.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [108]:
SKIP_TESTS = False

if not SKIP_TESTS:
    import time
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took", end_time-start_time, "seconds to solve.\n")

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        if count < len(sudokus):
            break

Testing very_easy sudokus
This is very_easy sudoku number 0
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]]
0-0
0-1
0-2
0-3
0-4
0-5
0-6
0-7
0-8
1-0
1-1
1-2
1-3
1-4
1-5
1-6
1-7
1-8
2-0
2-1
2-2
2-3
2-4
2-5
2-6
2-7
2-8
3-0
3-1
3-2
3-3
3-4
3-5
3-6
3-7
3-8
4-0
4-1
4-2
4-3
4-4
4-5
4-6
4-7
4-8
5-0
5-1
5-2
5-3
5-4
5-5
5-6
5-7
5-8
6-0
6-1
6-2
6-3
6-4
6-5
6-6
6-7
6-8
7-0
7-1
7-2
7-3
7-4
7-5
7-6
7-7
7-8
8-0
8-1
8-2
8-3
8-4
8-5
8-6
8-7
8-8


AttributeError: 'constraint_solver' object has no attribute 'constraints'

## Submission Test
The following cell tests if your notebook is ready for submission. **You must not skip this step!**

Restart the kernel and run the entire notebook (Kernel → Restart & Run All). Now look at the output of the cell below. 

*If there is no output, then your submission is not ready.* Either your code is still running (did you forget to skip tests?) or it caused an error.

As previously mentioned, failing to follow these instructions can result in a grade of zero.

In [ ]:
import sys
import pathlib

fail = False;

if not SKIP_TESTS:
    fail = True;
    print("You must set the SKIP_TESTS constant to True in the cell above.")
    
p1 = pathlib.Path('./readme.txt')
p2 = pathlib.Path('./readme.md')
if not (p1.is_file() or p2.is_file()):
    fail = True;
    print("You must include a separate file called readme.txt or readme.md in your submission.")
    
p3 = pathlib.Path('./sudoku.ipynb')
if not p3.is_file():
    fail = True
    print("This notebook file must be named sudoku.ipynb")
    
if "sudoku_solver" not in dir():
    fail = True;
    print("You must include a function called sudoku_solver which accepts a numpy array.")
else: 
    sudoku = np.load("data/very_easy_puzzle.npy")[0]
    solution = np.load("data/very_easy_solution.npy")[0]

    if not np.array_equal(sudoku_solver(sudoku), solution):
        print("Warning:")
        print("Your sudoku_solver function does not correctly solve the first sudoku.")
        print()
        print("Your assignment is unlikely to get any marks from the autograder. While we will")
        print("try to check it manually to assign some partial credit, we encourage you to ask")
        print("for help on the forum or directly to a tutor.")
        print()
        print("Please use the readme file to explain your code anyway.")
    
if fail:
    print()
    sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
else:
    print("All checks passed. When you are ready to submit, upload the notebook and readme file to the")
    print("assignment page, without changing any filenames.")
    print()
    print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")

In [ ]:
# This is a TEST CELL. Do not delete or change.